In [3]:
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import *
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit import BasicAer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC

from time import time
import numpy as np
from functools import reduce

In [4]:
def read_data(datafile, labelindex):
    data_file = open(datafile)
    datalines = data_file.readlines()
    database = []
    labels = []
    all_labels = set()
    for data in datalines[1:]:
        feature_vector = list(data.strip().split(","))
        database.append(list(map(float, feature_vector[:labelindex] + feature_vector[labelindex+1:])))
        labels.append(feature_vector[labelindex])
        all_labels.add(feature_vector[labelindex])
    label_encodings = {d:i for i,d in enumerate(all_labels)}
    encoded_labels = [label_encodings[label] for label in labels]

    return np.array(database), np.array(encoded_labels)

In [5]:
class classify:        
    def __init__(self, datafile, labelindex):
        self.database, self.labels = read_data(datafile, labelindex)
        self.feature_dim = self.database.shape[1]
    
    def split(self, train_size=0.8, random_state=42):
        return train_test_split(self.database, self.labels, train_size=train_size, random_state=random_state)
    
    def qsvc(self, feature_map=None, seed=1376):
        if feature_map is None:
            feature_map = ZFeatureMap(self.feature_dim)

        kernel = QuantumKernel(feature_map=feature_map,
                                    quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                                    shots=1,
                                                    seed_simulator=seed,
                                                    seed_transpiler=seed))
        qsvc1 = QSVC(quantum_kernel=kernel)
        self.classifier = qsvc1

    def csvc(self, kernel="rbf", gamma="scale", seed=1376):
        csvc1 = SVC(kernel=kernel, gamma=gamma, random_state=seed)
        self.classifier = csvc1

    def train(self, train_data, train_labels):
        start = time()
        self.classifier.fit(train_data, train_labels)
        end = time()
        print("Time taken to train: ", end-start)
    
    def predict(self, test_data):
        start = time()
        pred_labels = self.classifier.predict(test_data)
        end = time()
        print("Time taken to predict: ", end-start)
        return pred_labels
    
    def get_scores(self, test_labels, pred_labels):
        acc =  accuracy_score(test_labels, pred_labels)
        prec = precision_score(test_labels, pred_labels, average="macro")
        recall = recall_score(test_labels, pred_labels, average="macro")
        f1 = f1_score(test_labels, pred_labels, average="macro")

        print("Accuracy: ", acc)
        print("Precision: ", prec)
        print("Recall: ", recall)
        print("F1: ", f1)

        return acc, prec, recall, f1

In [19]:
def eq_8(x):
    return np.pi*reduce(lambda a,b: a*b, x)

def eq_9(x):
    return (np.pi/2)*reduce(lambda a,b: (1-a)*(1-b), x)

def eq_10(x):
    return np.pi*reduce(lambda a,b: np.exp(np.square(a-b)/8), x)

def eq_11(x):
    return (np.pi/3)*reduce(lambda a,b: 1/(np.cos(a)*np.cos(b)), x)

def eq_12(x):
    return np.pi*reduce(lambda a,b: np.cos(a)*np.cos(b), x)

In [21]:
def eq_8_1(x):
    phi = x[0] if len(x) == 1 else 0
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            phi += x[i]*x[j]
    return (np.pi/2)*phi

def eq_8_2(x):
    phi = x[0] if len(x) == 1 else x[0]*x[1]
    return np.pi*phi

def eq_9_1(x):
    phi = x[0] if len(x) == 1 else 0
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            phi += (1-x[i])*(1-x[j])
    return (np.pi/2)*phi

def eq_10_1(x):
    phi = x[0] if len(x) == 1 else 0
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            phi += np.exp(np.square(x[i]-x[j])/8)
    return np.pi*phi

def eq_11_1(x):
    phi = x[0] if len(x) == 1 else 0
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            phi += 1/(np.cos(x[i])*np.cos(x[j]))
    return (np.pi/3)*phi

def eq_12_1(x):
    phi = x[0] if len(x) == 1 else 0
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            phi += np.cos(x[i])*np.cos(x[j])
    return np.pi*phi    

In [ ]:
qc = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc.split()
qc.qsvc()
qc.train(train_data, train_labels)
pred_labels = qc.predict(test_data)
qc.print_scores(test_labels, pred_labels)

In [ ]:
qc1 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc1.split()
fm1 = ZFeatureMap(qc1.feature_dim, data_map_func=eq_8)
qc1.qsvc()
qc1.train(train_data, train_labels)
pred_labels = qc1.predict(test_data)
qc1.print_scores(test_labels, pred_labels)

In [ ]:
qc2 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc2.split()
fm2 = ZFeatureMap(qc2.feature_dim, data_map_func=eq_9)
qc2.qsvc()
qc2.train(train_data, train_labels)
pred_labels = qc2.predict(test_data)
qc2.print_scores(test_labels, pred_labels)

In [ ]:
qc3 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc3.split()
fm3 = ZFeatureMap(qc3.feature_dim, data_map_func=eq_10)
qc3.qsvc()
qc3.train(train_data, train_labels)
pred_labels = qc3.predict(test_data)
qc3.print_scores(test_labels, pred_labels)

In [ ]:
qc4 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc4.split()
fm4 = ZFeatureMap(qc4.feature_dim, data_map_func=eq_11)
qc4.qsvc()
qc4.train(train_data, train_labels)
pred_labels = qc4.predict(test_data)
qc4.print_scores(test_labels, pred_labels)

In [ ]:
qc5 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = qc5.split()
fm5 = ZFeatureMap(qc5.feature_dim, data_map_func=eq_12)
qc5.qsvc()
qc5.train(train_data, train_labels)
pred_labels = qc5.predict(test_data)
qc5.print_scores(test_labels, pred_labels)

In [ ]:
cc = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = cc.split()
cc.csvc(kernel='rbf')
cc.train(train_data, train_labels)
pred_labels = cc.predict(test_data)
cc.print_scores(test_labels, pred_labels)

In [ ]:
cc1 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = cc1.split()
cc1.csvc(kernel='linear')
cc1.train(train_data, train_labels)
pred_labels = cc1.predict(test_data)
cc1.print_scores(test_labels, pred_labels)

In [ ]:
cc2 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = cc2.split()
cc2.csvc(kernel='poly')
cc2.train(train_data, train_labels)
pred_labels = cc2.predict(test_data)
cc2.print_scores(test_labels, pred_labels)

In [ ]:
cc3 = classify("HeartFailure_Dataset.csv", 12)
train_data, test_data, train_labels, test_labels = cc3.split()
cc3.csvc(kernel='sigmoid')
cc3.train(train_data, train_labels)
pred_labels = cc3.predict(test_data)
cc3.print_scores(test_labels, pred_labels)